In [22]:
!pip install kora

import kora.install.rdkit
import json  # lets us work with the json format
import requests  # allows Python to make web requests
import pandas as pd # analysis of tabular data
import numpy as np # numerical library
import matplotlib.pyplot as plt # plotting
import rdkit.Chem.AllChem as AllChem # rdkit. chemoinformatics
import rdkit.Chem as Chem
from rdkit import Chem, DataStructs
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from ipywidgets import interact # widgets
import ipywidgets as widgets

import os
from matplotlib import gridspec
from rdkit.Chem.Fingerprints import FingerprintMols


# All we need for clustering
from scipy.cluster.hierarchy import dendrogram, linkage


In [23]:
#Si tienes los datos en drive:
#from google.colab import drive
#drive.mount('/content/drive')

In [24]:
# The result of this code will be:
working_library=[]
with open('Results_merged_Chembl_Mol1_Betv1_clustering_input.smi','r') as file:
    for index,line in enumerate(file):
        if index<=1: # Kee the fist smile code as example
            print (index, line.split( ))

0 ['Smiles', 'Compound', 'RMSD', 'Model.ID', 'Energy', 'nRot']
1 ['C[C@@H](C(=O)N[C@H](CO)CC1CCCCC1)NC(=O)[C@H](c1cc(cc(c1)F)F)O', 'CHEMBL101537', '0.058263782', '1', '-10.8', '8']


In [25]:
working_library=[]
with open('Results_merged_Chembl_Mol1_Betv1_clustering_input.smi','r') as file:
    for index,line in enumerate(file):
        if 0<index<=1500: # Molecules we want (0 is omitted because the fist line (0) of file is the header, not SMILES code)
            mol=Chem.MolFromSmiles(line.split()[0]) # Converting SMILES codes into rdkit mol 
            mol.SetProp('_Name',line.split()[1]) # Adding the name for each molecule
            working_library.append(mol)

In [ ]:
Draw.MolsToGridImage(working_library,molsPerRow=6,subImgSize=(110,130),legends=[mol.GetProp('_Name') for mol in working_library])

In [ ]:
#from rdkit.Chem import MACCSkeys
#fps = [MACCSkeys.GenMACCSKeys(mol) for mol in working_library]
#fps = [AllChem.GetMorganFingerprint(mol,3) for mol in working_library]
#fps= [FingerprintMols.FingerprintMol(mol) for mol in working_library]
fps = [Chem.RDKFingerprint(mol) for mol in working_library]

In [ ]:
print(len(working_library))
print(len(fps))

1500
1500


In [ ]:
#DataStructs.FingerprintSimilarity(fps[0],fps[1])
DataStructs.DiceSimilarity(fps[0],fps[1])

0.5627938213566152

In [ ]:
size=len(working_library)
hmap=np.empty(shape=(size,size))
table=pd.DataFrame()
for index, i in enumerate(fps):
    for jndex, j in enumerate(fps):
        similarity=DataStructs.FingerprintSimilarity(i,j) #si es con Morgan, DataStructs.DiceSimilarity(i,j), si es de otra forma DataStructs.FingerprintSimilarity(i,j)
        hmap[index,jndex]=similarity
        table.loc[working_library[index].GetProp('_Name'),working_library[jndex].GetProp('_Name')]=similarity

In [ ]:
table.head(100) # just the first 100 values due to our table in 100 x 1499

In [ ]:
#Define clustering setup
def ClusterFps(fps,cutoff=0.2):
    from rdkit import DataStructs
    from rdkit.ML.Cluster import Butina

    # first generate the distance matrix:
    dists = []
    size=len(working_library)
    hmap=np.empty(shape=(size,size))
    table=pd.DataFrame()
    for index, i in enumerate(fps):
      for jndex, j in enumerate(fps):
        similarity=DataStructs.DiceSimilarity(i,j)
        hmap[index,jndex]=similarity
        table.loc[working_library[index].GetProp('_Name'),working_library[jndex].GetProp('_Name')]=similarity
    
    nfps = len(fps)
    for i in range(1,nfps):
        sims = DataStructs.BulkTanimotoSimilarity(fps[i],fps[:i])
        dists.extend([1-x for x in sims])

    # now cluster the data:
    cs = Butina.ClusterData(dists,nfps,cutoff,isDistData=True)
    return cs

In [ ]:
clusters=ClusterFps(fps,cutoff=0.4)

In [ ]:
table.to_csv('TanimotoResults_Good.csv', index=False, sep=',')

In [ ]:
linked = linkage(hmap,'single')
labelList = [mol.GetProp('_Name') for mol in working_library]

In [ ]:
plt.figure(figsize=(25,200))

ax1=plt.subplot()
o=dendrogram(linked,  
            orientation='left',
            labels=labelList,
            distance_sort='descending',
            show_leaf_counts=True)

ax1.spines['left'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
plt.title('Similarity clustering',fontsize=20,weight='bold')
plt.tick_params ('both',width=2,labelsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# This will give us the clusters in order as the last plot
new_data=list(reversed(o['ivl']))

# we create a new table with the order of HCL
hmap_2=np.empty(shape=(size,size))
for index,i in enumerate(new_data):
    for jndex,j in enumerate(new_data):
        hmap_2[index,jndex]=table.loc[i].at[j]

In [ ]:
figure= plt.figure(figsize=(30,30))
gs1 = gridspec.GridSpec(2,7)
gs1.update(wspace=0.01)
ax1 = plt.subplot(gs1[0:-1, :2])
dendrogram(linked, orientation='left', distance_sort='descending',show_leaf_counts=True,no_labels=True)
ax1.spines['left'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax2 = plt.subplot(gs1[0:-1,2:6])
f=ax2.imshow (hmap_2, cmap='PRGn_r', interpolation='nearest')

ax2.set_title('Fingerprint Similarity',fontsize=20,weight='bold')
ax2.set_xticks (range(len(new_data)))
ax2.set_yticks (range(len(new_data)))
ax2.set_xticklabels (new_data,rotation=90,size=8)
ax2.set_yticklabels (new_data,size=8)

ax3 = plt.subplot(gs1[0:-1,6:7])
m=plt.colorbar(f,cax=ax3,shrink=0.75,orientation='vertical',spacing='uniform',pad=0.01)
m.set_label ('Fingerprint Similarity')

plt.tick_params ('both',width=2)
plt.plot()